# Training Analysis Project For Code Louisville Summer 2020 Program

## Import pandas and CSV file
Import pandas and read the Samplecomments2 CSV file, storing it in df

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('Samplecomments2.csv')

## Take a look at the data in the CSV 
Use: 
- .head to see the data in a spreadsheet format
- .shape to see how many rows/columns there are

## Look at how people responded to the "Receiving" question
Take a look at how many people said they are receiving what they need to train their people on promotions versus the number of people who do not.

## Examine the OSC scores compared to the "Receiving" response
Print:
- Average OSC score of those who said no vs. yes
- Median OSC score of no vs.  yes

## Look at responses to the "Wish" question
Keeping in mind that people can select multiple wishes, analyze:
- How many people selected shorter training
- Splitting up every entry and stack so each individual item can be counted; then, check the count for each